# RAG агент на базе YDB и YandexGPT
В данном документе будет создан простой RAG агент с использованием библиотеки LangChain, YDB как векторного хранилища, а также YandexGPT в качестве языковой модели

## Подготовка окружения

Установка необходимых зависимостей

In [1]:
!pip install -qU yandexcloud langchain-community langchain-ydb python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Загрузка переменных окружения из файла `.env`

В нем должны быть объявлены переменные:

`YC_FOLDER_ID` - id каталога yandexcloud
`YC_IAM_TOKEN` - IAM ключ пользователя, созданного в каталоге yandexcloud
`SA_KEY_FILE` - файл с ключом SA

In [2]:
import os
from dotenv import load_dotenv
dotenv_path = os.path.join(os.path.dirname(__name__), '.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

FOLDER_ID=os.environ.get('YC_FOLDER_ID')

In [13]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.schema import Document
import os

# Путь к папке с .md файлами
docs_path = "docs"

# Загружаем все .md файлы
loader = DirectoryLoader(
    path=docs_path,
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

raw_documents = loader.load()

print(len(raw_documents))

17


In [14]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Header1"), ("##", "Header2")])

split_docs = []
for doc in raw_documents:
    split_docs.extend(splitter.split_text(doc.page_content))

len(split_docs)

202

In [15]:
split_docs[100]

Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 66: Можно ли принимать препарат стоя на мелководье?'}, page_content='**Ответ:** Да, но эффект будет востребован только при полном погружении головы.  \n---')

## Создание Vector Store

Для создания YDB Vector Store потребуется embedding - функция превращения документов в вектор. В нашем случае будет использован `YandexGPTEmbeddings` - он хорошо работает с русским языком

In [16]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain_ydb.vectorstores import YDB, YDBSettings
import ydb.iam


embeddings = YandexGPTEmbeddings(folder_id=FOLDER_ID)

config = YDBSettings(
    host="lb.etnok7cd0an3kg6eshud.ydb.mdb.yandexcloud.net",
    port=2135,
    database="/ru-central1/b1g8skpblkos03malf3s/etnok7cd0an3kg6eshud",
    secure=True,
    table="ydb_rag_example_yagpt",
    drop_existing_table=True,
)

vector_store = YDB(
    embeddings,
    config=config,
    credentials=ydb.iam.ServiceAccountCredentials.from_file(
        os.getenv("SA_KEY_FILE"),
    )
)

Загрузка подготовленных документов в векторное хранилище YDB 

In [17]:
ids = vector_store.add_documents(split_docs, batch_size=1)
len(ids)

Processing batches...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [00:29<00:00,  6.88it/s]


202

## Поиск данных

Векторное хранилище можно использовать для поиска близких по смыслу и содержанию документов из пользовательского запроса. Ниже представлены примеры как YDB справляется с этой задачей.

In [19]:
vector_store.similarity_search("Что такое гидродых?")

[Document(metadata={'Header1': 'Instructions for Use'}, page_content='Гидродых — препарат, позволяющий человеку дышать под водой за счёт высвобождения кислорода в кровоток. Его использование требует строгого соблюдения инструкций для обеспечения максимальной эффективности и безопасности.'),
 Document(metadata={'Header1': 'Composition and Action'}, page_content='Гидродых — это результат многолетних исследований компании АКВАЭРА, направленных на создание безопасного и эффективного способа дыхания под водой. Препарат представляет собой таблетку, которая запускает в организме человека уникальные биохимические процессы, позволяющие насыщать кровь кислородом без участия дыхательных органов.'),
 Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 6: Нужно ли использовать акваланг вместе с Гидродых?'}, page_content='**Ответ:** Нет. Гидродых позволяет дышать под водой без дополнительного оборудования.  \n---'),
 Document(metadata={'Header1': 'Success Stories', 'Header2': 'Военн

In [21]:
vector_store.similarity_search("как препарат влияет на экологию?")

[Document(metadata={'Header1': 'Eco Strategy', 'Header2': 'Принципы экологической ответственности'}, page_content='АКВАЭРА руководствуется тремя ключевыми принципами:  \n1. **Минимизация воздействия на окружающую среду** – при разработке и тестировании новых препаратов учитывается влияние на флору и фауну.\n2. **Снижение углеродного следа** – производство оптимизировано для сокращения выбросов CO₂.\n3. **Разработка программ восстановления экосистем** – совместные проекты с научными институтами по оздоровлению рифов и прибрежных зон.'),
 Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 8: Что будет, если скормить Гидродых аквариумным рыбкам?'}, page_content='**Ответ:** Ничего хорошего. Препарат не предназначен для водных организмов и может нанести им вред.  \n---'),
 Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 39: Можно ли использовать Гидродых в гидромассажной ванне?'}, page_content='**Ответ:** Польза сомнительна, но безопасно. Главное — не 

In [22]:
vector_store.similarity_search("какие есть противопоказания?")

[Document(metadata={'Header1': 'Instructions for Use', 'Header2': 'Противопоказания'}, page_content='- Возраст до 16 лет\n- Беременность и период лактации\n- Хроническая гипероксигенация в анамнезе\n- Тяжёлые заболевания почек и печени\n- Аллергия на компоненты препарата'),
 Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 87: Можно ли принимать Гидродых вместе с антидепрессантами?'}, page_content='**Ответ:** Не рекомендуется без консультации с лечащим врачом.  \n---'),
 Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 41: Можно ли использовать Гидродых при занятиях подводным фитнесом?'}, page_content='**Ответ:** Да, при условии умеренной физической нагрузки и консультации с врачом перед использованием.  \n---'),
 Document(metadata={'Header1': 'Вопросы и ответы', 'Header2': 'Вопрос 98: Можно ли принимать Гидродых вместе с витаминами?'}, page_content='**Ответ:** Да, если они не влияют на дыхательную и сердечно-сосудистую системы.  \n---')]

## Сборка RAG агента

Найденные данные из YDB до возвращения пользователю можно отдать в LLM и попросить перефразировать. Ниже будет построен такой агент.

Схема итогового агента:

<div>
<img src="img/chain.png" width="800"/>
</div>


В качестве LLM будет использован YandexGPT, потому что он, опять же, хорошо справляется с русским языком.

In [23]:
from langchain_community.chat_models import ChatYandexGPT

chat_model = ChatYandexGPT(folder_id=FOLDER_ID)

In [24]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

# Явное создание шаблонов для SystemMessage и HumanMessage
system_template = """Ты - официальный представитель компании АКВАЭРА, которая разработала таблетки "Гидродых" для дыхания под водой.
Твоя задача - ответить на вопрос пользователя, опираясь на внутренние документы компании.
Если предложенный контекст не связан с пользовательским запросом - не фантазируй, попроси пользователя направить этот вопрос письмом на адрес юр лица".
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Контекст: {context}\n\nВопрос: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Объединение шаблонов сообщений в ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vector_store.as_retriever()

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | chat_model
    | StrOutputParser()
)

## Демонстрация работы RAG агента

In [25]:
rag_chain.invoke("что такое гидродых?")

'Гидродых — это препарат, разработанный компанией АКВАЭРА, который позволяет человеку дышать под водой за счёт высвобождения кислорода в кровоток. Он запускает в организме уникальные биохимические процессы, позволяющие насыщать кровь кислородом без участия дыхательных органов.'

In [27]:
rag_chain.invoke("какой у препарата срок годности?")

'Срок годности препарата — 18 месяцев с даты производства.'

In [28]:
rag_chain.invoke("Можно ли принимать гидродых дома в ванной?")

'Препарат «Гидродых» предназначен для использования под водой в условиях подводного плавания. Применение таблеток «Гидродых» в бытовых условиях, например, в ванной, не предусмотрено инструкцией и может быть опасно для здоровья. Перед применением препарата необходимо ознакомиться с инструкцией и убедиться в отсутствии противопоказаний.'

In [26]:
rag_chain.invoke("сколько стоит одна упаковка?")

'К сожалению, в предоставленных внутренних документах компании АКВАЭРА нет информации о стоимости одной упаковки таблеток «Гидродых». Пожалуйста, направьте ваш вопрос письмом на адрес юридического лица.'